<a href="https://colab.research.google.com/github/ChrisW2420/FedPKDG/blob/main/FedPKDG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FedPKDG -- Prune + KD + GAN + FL
This prototype implements the algorithm in a distributed setting
TODO:
1. implement a FedAvg aggregator/server
2. build a centralised FL system with n clients connected to the server
3. design experiments to assess accuracy, efficiency, generalisation on homogenoeous data
4. repeat experiments on heterogeneous data, identical model sparsity
5. repeat experiments on heterogeneous data, different model sparsity, mimicing different computational capability of clients

# Setup and Imports

In [1]:
# NB: package versions are very important
!pip install -q tensorflow-model-optimization # for pruning
!pip install -q git+https://github.com/tensorflow/docs # newest tf
!pip install --upgrade keras #newest keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [2]:
# 3 versions of keras are used for different functionalities, imported as different names
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
import tf_keras as keras_model #only for pruning
from tf_keras import layers as model_layers
import keras
import tempfile
from tf_keras.callbacks import EarlyStopping, Callback
from keras import ops, layers
from tensorflow_docs.vis import embed # for GAN
import matplotlib.pyplot as plt

In [ ]:
# Logging metrics with WandB
!pip install wandb
import wandb
wandb.login()
from wandb.keras import WandbMetricsLogger

## Loading Data

In [3]:
# MNIST
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Components Implementation

## Model zoo

### CNN

In [4]:
def miniCNN():
  model = keras_model.Sequential(
      [
          keras_model.Input(shape=(28, 28, 1)),
          model_layers.Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
          model_layers.Flatten(),
          model_layers.Dense(10),
      ],
      name="minicnn",
  )
  return model

def smallCNN():
  model = keras_model.Sequential(
      [
          keras_model.Input(shape=(28, 28, 1)),
          model_layers.Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
          model_layers.LeakyReLU(alpha=0.2),
          model_layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
          model_layers.Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
          model_layers.Flatten(),
          model_layers.Dense(10),
      ],
      name="smallcnn",
  )
  return model

def mediumCNN():
  model = keras_model.Sequential(
      [
          keras_model.Input(shape=(28, 28, 1)),
          model_layers.Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
          model_layers.LeakyReLU(alpha=0.2),
          model_layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
          model_layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
          model_layers.LeakyReLU(alpha=0.2),
          model_layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
          model_layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
          model_layers.Flatten(),
          model_layers.Dense(10),
      ],
      name="mediumcnn",
  )
  return model

def bigCNN():
  model = keras_model.Sequential(
      [
          keras_model.Input(shape=(28, 28, 1)),
          model_layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
          model_layers.LeakyReLU(alpha=0.2),
          model_layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
          model_layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
          model_layers.LeakyReLU(alpha=0.2),
          model_layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
          model_layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
          model_layers.Flatten(),
          model_layers.Dense(10),
      ],
      name="bigcnn",
  )
  return model

### GAN

In [5]:
num_channels = 1
num_classes = 10
image_size = 28
latent_dim = 128

generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes

# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((28, 28, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(7 * 7 * generator_in_channels),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

## CNN (don't run for now)

In [ ]:
# very unecessary in this class itself, might be useful to combined pruning and KD and ordinary training
class CNN(): #keras_model.Model
  def __init__(self, config_name, **kwargs):
    #super(CNN, self).__init__(**kwargs)
    if config_name == 'small':
      self.model = smallCNN()
    elif config_name == 'medium':
      self.model = mediumCNN()
    elif config_name == 'big':
      self.model = bigCNN()
    else:
      print('default model of medium CNN')
      self.model = mediumCNN()
    self.optimizer = 'adam'
    self.task_loss = keras_model.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.metricslist = [keras_model.metrics.SparseCategoricalAccuracy()]
    self.validation_split = 0.1
    self.early_stopping = EarlyStopping(
      monitor='val_loss',
      min_delta=0.001,  # only consider as improvement significant changes
      patience=2,      # number of epochs with no improvement after which training will be stopped
      verbose=1,
      mode='min'        # 'min' because we want to minimize the loss
    )
    self.callbacks = []


  def train(self, training_data, testing_data = None, epochs = 10, is_earlystop = True, **kwargs):
    #super(CNN, self).compile(optimizer=self.optimizer, metrics=self.metricslist, **kwargs)
    self.model.compile(
      optimizer=self.optimizer,
      loss=self.task_loss,
      metrics=self.metricslist
    )
    if is_earlystop and self.early_stopping not in self.callbacks:
      self.callbacks.append(self.early_stopping)
    x_train, y_train = training_data
    self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,validation_split=self.validation_split, callbacks=self.callbacks)
    if testing_data:
      x_test, y_test = testing_data
      self.model.evaluate(x_test, y_test)
    return self.model


## GAN

In [6]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.seed_generator = keras.random.SeedGenerator(1337)
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = ops.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = ops.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = ops.shape(real_images)[0]
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim), seed=self.seed_generator
        )
        random_vector_labels = ops.concatenate(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = ops.concatenate(
            [generated_images, image_one_hot_labels], -1
        )
        real_image_and_labels = ops.concatenate([real_images, image_one_hot_labels], -1)
        combined_images = ops.concatenate(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = ops.concatenate(
            [ops.ones((batch_size, 1)), ops.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim), seed=self.seed_generator
        )
        random_vector_labels = ops.concatenate(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = ops.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = ops.concatenate(
                [fake_images, image_one_hot_labels], -1
            )
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [7]:
# image generation functions
def generate_image(generator, target_class, latent_dim):
    noise_matrix = keras.random.normal(shape=(1, latent_dim))
    # Convert the target label to one-hot encoded vectors.
    target_label = keras.utils.to_categorical([target_class], num_classes)
    target_label = ops.cast(target_label, "float32")
    noise_and_labels = ops.concatenate([noise_matrix, target_label], 1)
    fake = generator.predict(noise_and_labels,verbose = 0)
    return fake

def pseudoDataset(generator, total_num, latent_dim): # producing equal numbers of samples for each class
    pseudo_images = []
    for num in range(10):
      target_class = num
      print('Generating', int(total_num/10), 'fake images of digit', num, '......')
      for _ in range(int(total_num/10)):
        generated_images = generate_image(target_class, latent_dim)
        generated_images *= 255.0
        converted_images = generated_images.astype(np.uint8)
        converted_images = ops.image.resize(converted_images, (28, 28)).numpy().astype(np.uint8)
        pseudo_images.append(converted_images)
    pseudo_images = np.concatenate(pseudo_images, axis=0)
    pseudo_labels = np.repeat(np.arange(10), int(total_num/10))
    return pseudo_images, pseudo_labels

## Pruning

In [8]:
def prune_finetrain(base_model, _epochs, x, y, validation_split=0.1, target_sparsity = 0.5, fine_tune_epochs=0):
  callbacks = [
      sparsity.UpdatePruningStep(),
      early_stopping
  ]
  steps_per_epoch = len(x)*(1-validation_split) // batch_size
  begin_step=int(steps_per_epoch*fine_tune_epochs)
  end_step=int(steps_per_epoch*_epochs)+1
  print('begin_step=', begin_step, 'end_step=', end_step)
  pruning_schedule = sparsity.PolynomialDecay(initial_sparsity=0, final_sparsity=target_sparsity,
                                              begin_step=begin_step, end_step=end_step) # TODO: tune begin_step, consider fining training before starting to prune

  model_for_pruning = sparsity.prune_low_magnitude(base_model, pruning_schedule=pruning_schedule) #default constant sparsity of 50%

  model_for_pruning.compile(
        optimizer='adam',
        loss=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras_model.metrics.SparseCategoricalAccuracy()]
  )

  model_for_pruning.fit(
      x,
      y,
      batch_size=batch_size,
      validation_split=validation_split,
      callbacks=callbacks,
      epochs=_epochs,
  )
  pruned_model = sparsity.strip_pruning(model_for_pruning)

  return pruned_model


# Model size metrics

def get_model_sparsity(model):
    total_weights = 0
    zero_weights = 0
    for weight in model.get_weights():
        total_weights += weight.size
        zero_weights += np.count_nonzero(weight == 0)
    return zero_weights / total_weights

def get_gzipped_model_size(model):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, keras_file = tempfile.mkstemp('.h5')
  model.save(keras_file, include_optimizer=False)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)

  return os.path.getsize(zipped_file)

## Knowledge Distillation

In [9]:
# Distiller class from before
class Distiller(keras_model.Model):
    def __init__(self, teacher, student, alpha=0.1, temperature=3, **kwargs):
        super(Distiller, self).__init__(**kwargs)
        self.teacher = teacher
        self.student = student

    def compile(self, optimizer, metrics, student_loss_fn, distillation_loss_fn, alpha, temperature, **kwargs):
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics, **kwargs)
        self.student_loss_fn = student_loss_fn
        self.student.compile(optimizer=optimizer, metrics=metrics, loss=self.student_loss_fn)
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack the data
        x, y = data

        # Forward pass of teacher with no gradient tracking
        teacher_predictions = self.teacher(x, training=False)

        print('testing kd train step')
        print(teacher_predictions)

        with tf.GradientTape() as tape:
            # Forward pass of the student
            student_predictions = self.student(x, training=True)

            # Calculate the task-specific loss
            task_loss = self.student_loss_fn(y, student_predictions)

            # Calculate the soft targets and the distillation loss
            soft_targets = tf.nn.softmax(teacher_predictions / self.temperature)

            print('soft target:', soft_targets)
            student_soft = tf.nn.softmax(student_predictions / self.temperature)
            distillation_loss = self.distillation_loss_fn(soft_targets, student_soft)

            # Calculate the total loss
            total_loss = (1 - self.alpha) * task_loss + self.alpha * distillation_loss * (self.temperature ** 2)
            print('total_loss:', total_loss)

        # Compute gradients and update weights
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(total_loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update metrics
        self.compiled_metrics.update_state(y, student_predictions)
        results = {m.name: m.result() for m in self.metrics}
        results.update({"task_loss": task_loss, "distillation_loss": distillation_loss, "total_loss": total_loss})
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Forward pass of the student
        y_pred = self.student(x, training=False)

        # Calculate the task-specific loss
        task_loss = self.student_loss_fn(y, y_pred)

        # Update the metrics
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def call_model(self):
      return self.student

In [14]:
teacher = mediumCNN()
teacher.compile(
        optimizer='adam',
        loss=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras_model.metrics.SparseCategoricalAccuracy()]
      )
teacher.fit(x_train, y_train, epochs=10,validation_split=0.1, callbacks=[early_stopping])

Epoch 1/10
1688/1688 [==============================] - 65s 11ms/step - loss: 0.2946 - sparse_categorical_accuracy: 0.9080 - val_loss: 0.0943 - val_sparse_categorical_accuracy: 0.9730
Epoch 2/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1005 - sparse_categorical_accuracy: 0.9683 - val_loss: 0.0713 - val_sparse_categorical_accuracy: 0.9795
Epoch 3/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0779 - sparse_categorical_accuracy: 0.9755 - val_loss: 0.0683 - val_sparse_categorical_accuracy: 0.9805
Epoch 4/10
1688/1688 [==============================] - 21s 13ms/step - loss: 0.0684 - sparse_categorical_accuracy: 0.9782 - val_loss: 0.0567 - val_sparse_categorical_accuracy: 0.9850
Epoch 5/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0601 - sparse_categorical_accuracy: 0.9801 - val_loss: 0.0537 - val_sparse_categorical_accuracy: 0.9862
Epoch 6/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.

In [15]:
raw = miniCNN()
raw.compile(optimizer=keras_model.optimizers.Adam(),
            metrics=[keras_model.metrics.SparseCategoricalAccuracy()],
            loss=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),)
raw.fit(x_train, y_train, epochs=3, validation_split=0.1)
raw.evaluate(x_test, y_test)

Epoch 1/3
1688/1688 [==============================] - 9s 4ms/step - loss: 0.3918 - sparse_categorical_accuracy: 0.8894 - val_loss: 0.2513 - val_sparse_categorical_accuracy: 0.9267
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2994 - sparse_categorical_accuracy: 0.9142 - val_loss: 0.2327 - val_sparse_categorical_accuracy: 0.9362
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.2713 - sparse_categorical_accuracy: 0.9216


[0.27132585644721985, 0.9215999841690063]

In [16]:
# using class above
student = miniCNN()

distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras_model.optimizers.Adam(),
    metrics=[keras_model.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras_model.losses.KLDivergence(),
    alpha=0.4,
    temperature=3,
)

# Distill teacher to student

distiller.fit(x_train, y_train, epochs=3, validation_split=0.1)
distiller.evaluate(x_test, y_test)

Epoch 1/3
testing kd train step
Tensor("mediumcnn/dense_1/BiasAdd:0", shape=(None, 10), dtype=float32)
soft target: Tensor("Softmax:0", shape=(None, 10), dtype=float32)
total_loss: Tensor("add:0", shape=(), dtype=float32)
testing kd train step
Tensor("mediumcnn/dense_1/BiasAdd:0", shape=(None, 10), dtype=float32)
soft target: Tensor("Softmax:0", shape=(None, 10), dtype=float32)
total_loss: Tensor("add:0", shape=(), dtype=float32)
1688/1688 [==============================] - 15s 7ms/step - sparse_categorical_accuracy: 0.8799 - task_loss: 0.4605 - distillation_loss: 0.3362 - total_loss: 1.4865 - val_sparse_categorical_accuracy: 0.9262
Epoch 2/3
1688/1688 [==============================] - 12s 7ms/step - sparse_categorical_accuracy: 0.9077 - task_loss: 0.3734 - distillation_loss: 0.2436 - total_loss: 1.1009 - val_sparse_categorical_accuracy: 0.9265
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - sparse_categorical_accuracy: 0.9179


0.917900025844574

In [18]:
#using class below
student = miniCNN()

distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras_model.optimizers.Adam(),
    metrics=[keras_model.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras_model.losses.KLDivergence(),
    alpha=0.4,
    temperature=3,
)

# Distill teacher to student

distiller.fit(x_train, y_train, epochs=3, validation_split=0.1)
distiller.evaluate(x_test, y_test)

Epoch 1/3
testing kd train step
testing kd train step
1688/1688 [==============================] - 16s 9ms/step - distillation_loss: 0.3307 - val_sparse_categorical_accuracy: 0.9205 - val_distillation_loss: 0.2227
Epoch 2/3
1688/1688 [==============================] - 16s 9ms/step - distillation_loss: 0.2435 - val_sparse_categorical_accuracy: 0.9280 - val_distillation_loss: 0.2102
Epoch 3/3
313/313 [==============================] - 3s 8ms/step - sparse_categorical_accuracy: 0.9129 - distillation_loss: 0.2292


[0.9128999710083008, 0.2291950285434723]

In [17]:
# copied
class Distiller(keras_model.Model):
    def __init__(self, teacher, student, alpha=0.1, temperature=3, **kwargs):
        super().__init__(**kwargs)
        self.teacher = teacher
        self.student = student
        self.loss_tracker = keras.metrics.Mean(name="distillation_loss")

    @property
    def metrics(self):
        metrics = super().metrics
        metrics.append(self.loss_tracker)
        return metrics

    def compile(self, optimizer, metrics, student_loss_fn, distillation_loss_fn, alpha, temperature, **kwargs):
        super().compile(optimizer=optimizer, metrics=metrics, **kwargs)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack the data
        x, y = data

        # Forward pass of teacher with no gradient tracking
        teacher_predictions = self.teacher(x, training=False)

        print('testing kd train step')

        with tf.GradientTape() as tape:
            # Forward pass of the student
            student_predictions = self.student(x, training=True)

            # Calculate the task-specific loss
            task_loss = self.student_loss_fn(y, student_predictions)

            # Calculate the soft targets and the distillation loss
            soft_targets = tf.nn.softmax(teacher_predictions / self.temperature, axis=1)
            student_soft = tf.nn.softmax(student_predictions / self.temperature, axis=1)
            distillation_loss = self.distillation_loss_fn(soft_targets, student_soft)

            # Calculate the total loss
            total_loss = (1 - self.alpha) * task_loss + self.alpha * distillation_loss * (self.temperature ** 2)

        # Compute gradients and update weights
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(total_loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Report Progress
        self.loss_tracker.update_state(distillation_loss)
        return {"distillation_loss": self.loss_tracker.result()}

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Forward passes
        teacher_predictions = self.teacher(x, training=False)
        student_predictions = self.student(x, training=False)

        # Calculate the loss
        distillation_loss = self.distillation_loss_fn(
            tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
            tf.nn.softmax(student_predictions / self.temperature, axis=1),
        )
        # Calculate the task-specific loss
        task_loss = self.student_loss_fn(y, student_predictions)

        self.loss_tracker.update_state(distillation_loss)
        self.compiled_metrics.update_state(y, student_predictions)
        results = {m.name: m.result() for m in self.metrics}
        return results


In [67]:
for alpha in range(10):
  alpha = alpha/10
  student = miniCNN()
  distiller = Distiller(student=student, teacher=teacher)
  distiller.compile(
      optimizer=keras_model.optimizers.Adam(),
      metrics=[keras_model.metrics.SparseCategoricalAccuracy()],
      distillation_loss_fn=keras_model.losses.KLDivergence(),
      student_loss_fn=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),
      alpha=alpha,
      temperature=3,
  )
  distiller.fit(x_train, y_train, epochs=3, validation_split=0.1, verbose=0)
  print('alpha = ', alpha)
  distiller.evaluate(x_test, y_test)

testing kd train step
testing kd train step
alpha =  0.0
313/313 [==============================] - 1s 4ms/step - sparse_categorical_accuracy: 0.9506 - distillation_loss: 0.1844
testing kd train step
testing kd train step
alpha =  0.1
313/313 [==============================] - 1s 4ms/step - sparse_categorical_accuracy: 0.9525 - distillation_loss: 0.0985
testing kd train step
testing kd train step
alpha =  0.2
313/313 [==============================] - 2s 5ms/step - sparse_categorical_accuracy: 0.9639 - distillation_loss: 0.0551
testing kd train step
testing kd train step
alpha =  0.3
313/313 [==============================] - 1s 4ms/step - sparse_categorical_accuracy: 0.9480 - distillation_loss: 0.0855
testing kd train step
testing kd train step
alpha =  0.4
313/313 [==============================] - 2s 5ms/step - sparse_categorical_accuracy: 0.9681 - distillation_loss: 0.0401
testing kd train step
testing kd train step
alpha =  0.5
313/313 [==============================] - 1s 4ms/ste

In [64]:
raw = miniCNN()
raw.compile(optimizer=keras_model.optimizers.Adam(),
            metrics=[keras_model.metrics.SparseCategoricalAccuracy()],
            loss=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),)
raw.fit(x_train, y_train, epochs=3, validation_split=0.1)
raw.evaluate(x_test, y_test)

Epoch 1/3
1688/1688 [==============================] - 9s 4ms/step - loss: 0.3918 - sparse_categorical_accuracy: 0.8905 - val_loss: 0.2065 - val_sparse_categorical_accuracy: 0.9427
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2206 - sparse_categorical_accuracy: 0.9370 - val_loss: 0.1563 - val_sparse_categorical_accuracy: 0.9627
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - loss: 0.1502 - sparse_categorical_accuracy: 0.9568


[0.150151789188385, 0.9567999839782715]

# Helper Function Implementation

TODO:
Dataset:
- Dataloader
- heterogeneous dataset partition
- data augmentation

visualisation:
- dataset example visualisation
- data distribution visualisation
- confusion matrix
-

In [11]:
def set_model_weights_to_zero(model):
    for layer in model.layers:
        zero_weights = [np.zeros_like(w) for w in layer.get_weights()]
        layer.set_weights(zero_weights)
    return model

In [12]:
def if_synced(model1, model2):
    for layer1, layer2 in zip(model1.layers, model2.layers):
          weights1 = layer1.get_weights()
          weights2 = layer2.get_weights()
          for w1, w2 in zip(weights1, weights2):
              if not np.array_equal(w1, w2):
                  print('different weights, syncing failed')
    print('weights synced for client')

## Callback zoo

In [13]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,  # only consider as improvement significant changes
    patience=2,      # number of epochs with no improvement after which training will be stopped
    verbose=1,
    mode='min'        # 'min' because we want to minimize the loss
    )

# Client

In [ ]:
class Client(): #TODO: add name to clients to refer to them, espeically during logging
  def __init__(self, model_fn, x_train, y_train, generator = generator, discriminator = discriminator, **kwargs):
    self.cnn = model_fn
    self.generator = generator
    self.discriminator = discriminator
    self.latent_dim = 128 # hyperparam, can tune
    self.x_private = x_train
    self.y_private = y_train
    self.batch_size = 64 # hyperparam, can tune
    self.validation_split=0.1

  def local_train(self, epochs = 5, is_prune = False, sparsity = 0.5, fine_tune_epochs = 0, **kwargs):
    if is_prune:
      #!!!TODO: test prune
      print('after aggregating client has sparsity', get_model_sparsity(client.cnn))
      self.cnn = prune_finetrain(self.cnn, epochs, x = self.x_private, y = self.y_private, fine_tune_epochs = fine_tune_epochs) # fine_tune_epochs can take decimals, starts pruning after fine tune
      self.cnn.compile(
        optimizer='adam',
        loss=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras_model.metrics.SparseCategoricalAccuracy()]
      )
      print('before aggregating client has sparsity', get_model_sparsity(client.cnn))
    else:
      self.cnn.compile(
        optimizer='adam',
        loss=keras_model.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras_model.metrics.SparseCategoricalAccuracy()]
      )
      self.cnn.fit(self.x_private, self.y_private, batch_size=batch_size, epochs=epochs,validation_split=self.validation_split, callbacks=[early_stopping])

  # ! TODO: cnn evaluation def eval_cnn

  def train_gen(self, epochs = 20, d_learning_rate = 0.0003, g_learning_rate = 0.0003):
    #TODO: test ConditionalGAN
    cond_gan = ConditionalGAN(self.discriminator, self.generator, self.latent_dim)
    cond_gan.compile(
        d_optimizer=keras.optimizers.Adam(d_learning_rate),
        g_optimizer=keras.optimizers.Adam(g_learning_rate),
        loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
    )
    # produce GAN training dataset
    train_label = keras.utils.to_categorical(self.y_private, 10) # 1 hot encoding label
    dataset = tf.data.Dataset.from_tensor_slices((self.x_private, train_label))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
    cond_gan.fit(dataset, epochs)


  ## the following code act as interface with the server, avoid direct access to private model and dataset

  def produce_logits(self, dataset): #for KD
    #!!!TODO
    logits = []
    return logits

  def get_cnn_weights(self): #only for FedAvg, disabled
    return self.cnn.get_weights()

  def set_cnn_weights(self, weights): #for downloading global weights
    self.cnn.set_weights(weights)

  def get_gen_weights(self): #only for FedAvg, disabled
    return self.generator.get_weights()

  def set_gen_weights(self, weights): #for downloading global weights
    self.generator.set_weights(weights)

  def get_datasize(self):
    return self.x_private.shape[0]

# Server

In [ ]:
class Server():
  def __init__(self, model_fn, client_list, generator = generator, comm_freq = 1, algo = 'FedAvg', **kwargs):
    self.cnn = model_fn
    self.client_list = client_list # calling this param when "uploading" or "downloading"
    self.client_datasize = []
    self.generator = generator
    self.latent_dim = 128 # hyperparam, can tune
    self.public_dataset = None # to generate
    self.batch_size = 64 # hyperparam, can tune
    self.comm_freq = comm_freq # no. of client local training epochs before upload
    self.dataset = None

    # default settings for FedAvg
    self.is_prune = False

    if algo == 'FedPKDG':
    # turn on FedPKDG
      self.is_prune = True

  def get_client_datasize(self):
    if len(self.client_datasize) != len(self.client_list):
      for i in range(len(self.client_datasize), len(self.client_list)):
        self.client_datasize.append(self.client_list[i].get_datasize())
    return self.client_datasize

  def assign_weights_cnn(self, client):
    client.set_cnn_weights(self.cnn.get_weights())

  def assign_weights_gen(self, client):
    client.set_gen_weights(self.generator.get_weights())

  def broadcast(self):
    # TODO: improve: can use tff.federated_map and tff.federated_broadcast, can combine the two assign fns
    for client in self.client_list:
        self.assign_weights_cnn(client)
    for client in self.client_list:
        self.assign_weights_gen(client)

  def local_training(self):
    for idx, client in enumerate(self.client_list):
      # train cnn only
      print('training client', idx, '\'s cnn')
      client.local_train(epochs = self.comm_freq, is_prune = self.is_prune)
      # TODO: traingen

  # def agg_classifier(self):
  #   # !!TODO: can use tff.federated_mean
  #   for i in range(len(self.client_list)):
  #     # !!TODO: aggregate the classifier layer and freeze it

  def agg_cnn(self):
    global_weights = [np.zeros_like(w) for w in self.cnn.get_weights()]
    p = self.get_client_datasize()
    total_size = sum(p)
    for client_idx, client in enumerate(client_list):
        p_k = p[client_idx]/total_size
        client_weights = client.get_cnn_weights()
        for layer_idx, weight in enumerate(client_weights):
            global_weights[layer_idx] += p_k * weight
    # Set the updated weights to the global model
    self.cnn.set_weights(global_weights)

  def agg_gen(self):
    global_weights = [np.zeros_like(w) for w in self.generator.get_weights()]
    p = self.get_client_datasize()
    total_size = sum(p)
    for client_idx, client in enumerate(client_list):
        p_k = p[client_idx]/total_size
        client_weights = client.get_gen_weights()
        for layer_idx, weight in enumerate(client_weights):
            global_weights[layer_idx] += p_k * weight
    # Set the updated weights to the global model
    self.generator.set_weights(global_weights)

  def produce_pseudo_dataset(self, total_num):
    # generate with gen, homogenous data: equal number of datapoints for each class
    #TODO: test pseudoDataset
    self.dataset = pseudoDataset(self.generator, total_num, self.latent_dim)

  def agg_logits(self, datapoint):
    # mimics clients sending their logits to the server given the same input
    p = self.get_client_datasize()
    total_size = sum(p)
    for i in range(len(self.client_list)):
      client = self.client_list[i]
      p_k = p[i]/total_size
      if i == 0:
        logits = p_k * client.produce_logits(datapoint)
      else:
        logits += p_k * client.produce_logits(datapoint)
    return logits


  def distill(): ##HARD and IMPORTANT!
    #!!!TODO: distillation based on self.dataset and agg_logits
    return


# Testing Functionality

NB: re-run the server block before every experiment to avoid error: Server class not callable

## FedAvg (Don't Touch)

In [ ]:
#initiate 3 clients
no_sample = len(x_train) // 3
client_list = []
for i in range(3):
  #partition dataset to mimic private data
  x_train_k = x_train[no_sample*i:no_sample*(i+1)]
  y_train_k = y_train[no_sample*i:no_sample*(i+1)]
  client_list.append(Client(smallCNN(), x_train_k, y_train_k))

#initiate 1 server
Server = Server(smallCNN(), client_list, comm_freq = 1)

for _ in range(3):
  Server.broadcast()
  print('Broadcasted weights to all clients')
  Server.local_training()
  print('trained all clients cnn round', _)
  Server.agg_cnn()
  print('Weighted aggregated client weights')

for client in client_list:
  client.cnn.evaluate(x_test, y_test)

Broadcasted weights to all clients
training client 0 's cnn
282/282 [==============================] - 6s 14ms/step - loss: 0.8666 - sparse_categorical_accuracy: 0.7477 - val_loss: 0.3297 - val_sparse_categorical_accuracy: 0.9050
training client 1 's cnn
282/282 [==============================] - 5s 11ms/step - loss: 0.8512 - sparse_categorical_accuracy: 0.7609 - val_loss: 0.3672 - val_sparse_categorical_accuracy: 0.8970
training client 2 's cnn
282/282 [==============================] - 7s 19ms/step - loss: 0.8463 - sparse_categorical_accuracy: 0.7662 - val_loss: 0.2613 - val_sparse_categorical_accuracy: 0.9210
trained all clients cnn round 0
Weighted aggregated client weights
Broadcasted weights to all clients
training client 0 's cnn
282/282 [==============================] - 6s 14ms/step - loss: 0.3540 - sparse_categorical_accuracy: 0.8938 - val_loss: 0.2673 - val_sparse_categorical_accuracy: 0.9210
training client 1 's cnn
282/282 [==============================] - 6s 12ms/step - 

In [ ]:
x_train_k = x_train[:no_sample]
y_train_k = y_train[:no_sample]
client = Client(smallCNN(), x_train_k, y_train_k)
client.local_train(epochs=3)
client.cnn.evaluate(x_test, y_test)

Epoch 1/3
282/282 [==============================] - 5s 14ms/step - loss: 0.7805 - sparse_categorical_accuracy: 0.7734 - val_loss: 0.3458 - val_sparse_categorical_accuracy: 0.9010
Epoch 2/3
282/282 [==============================] - 3s 12ms/step - loss: 0.3653 - sparse_categorical_accuracy: 0.8892 - val_loss: 0.2914 - val_sparse_categorical_accuracy: 0.9185
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - loss: 0.2903 - sparse_categorical_accuracy: 0.9135


[0.29031461477279663, 0.9135000109672546]

comment: model quickly overfit to bad training data even after syncing weights, need to prevent this

## Just Pruning (Don't Touch)

In [ ]:
no_sample = len(x_train) // 3
client_list = []
for i in range(3):
  #partition dataset to mimic private data
  x_train_k = x_train[no_sample*i:no_sample*(i+1)]
  y_train_k = y_train[no_sample*i:no_sample*(i+1)]
  client_list.append(Client(smallCNN(), x_train_k, y_train_k))

#initiate 1 server
Server = Server(smallCNN(), client_list, comm_freq = 1, algo = 'FedPKDG')

for _ in range(3):
  Server.broadcast()
  print('Broadcasted weights to all clients')
  Server.local_training()
  print('trained all clients cnn round', _)
  Server.agg_cnn()
  print('Weighted aggregated client weights')

for client in client_list:
  get_model_sparsity(client.cnn)
  client.cnn.evaluate(x_test, y_test)

after broadcasting client has sparsity 0.0056595559425337396
after broadcasting client has sparsity 0.0056595559425337396
after broadcasting client has sparsity 0.0056595559425337396
Broadcasted weights to all clients
training client 0 's cnn
begin_step= 0 end_step= 282
282/282 [==============================] - 9s 13ms/step - loss: 0.8240 - sparse_categorical_accuracy: 0.7577 - val_loss: 0.3441 - val_sparse_categorical_accuracy: 0.8970
training client 1 's cnn
begin_step= 0 end_step= 282
282/282 [==============================] - 9s 18ms/step - loss: 0.8242 - sparse_categorical_accuracy: 0.7551 - val_loss: 0.4127 - val_sparse_categorical_accuracy: 0.8890
training client 2 's cnn
begin_step= 0 end_step= 282
282/282 [==============================] - 7s 12ms/step - loss: 0.8215 - sparse_categorical_accuracy: 0.7543 - val_loss: 0.2855 - val_sparse_categorical_accuracy: 0.9250
trained all clients cnn round 0
before agg client 0 has sparsity 0.4849804092294297
before agg client 1 has spars

In [ ]:
x_train_k = x_train[:no_sample]
y_train_k = y_train[:no_sample]
client = Client(smallCNN(), x_train_k, y_train_k)
client.local_train(epochs = 3, is_prune = True)
client.cnn.evaluate(x_test, y_test)

begin_step= 0 end_step= 844
Epoch 1/3
282/282 [==============================] - 7s 12ms/step - loss: 0.8385 - sparse_categorical_accuracy: 0.7484 - val_loss: 0.3486 - val_sparse_categorical_accuracy: 0.8985
Epoch 2/3
282/282 [==============================] - 4s 15ms/step - loss: 0.3786 - sparse_categorical_accuracy: 0.8857 - val_loss: 0.2972 - val_sparse_categorical_accuracy: 0.9150
Epoch 3/3
313/313 [==============================] - 2s 3ms/step - loss: 0.3094 - sparse_categorical_accuracy: 0.9068


[0.30936574935913086, 0.9067999720573425]

# Experiments